In [33]:
from tensorflow.keras.models import load_model

model = load_model('Ukrainian_OCR_tf_2.1.h5')

In [34]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [35]:
open('Ukrainian_OCR_tf_2.1.tflite', 'wb').write(tflite_model)

1373168

In [36]:
import numpy as np

def load_dataset(datasetPath):

  # List for storing data
  data = []
  
  # List for storing labels
  labels = []
  
  for row in open(datasetPath): #Openfile and start reading each row
    #Split the row at every comma
    row = row.split(",")
    
    #row[0] contains label
    label = int(row[0])
    
    #Other all collumns contains pixel values make a saperate array for that
    image = np.array([int(x) for x in row[1:]], dtype="uint8")
    
    #Reshaping image to 28 x 28 pixels
    image = image.reshape((32, 32))
    
    #append image to data
    data.append(image)
    
    #append label to labels
    labels.append(label)
    
  #Converting data to numpy array of type float32
  data = np.array(data, dtype='float32')
  
  #Converting labels to type int
  labels = np.array(labels, dtype="int")
  
  return (data, labels)

In [37]:
def get_batch_predictions(interpreter, input, output, batch):
  interpreter.set_tensor(input[0]['index'], batch)
  interpreter.invoke()
  predictions = interpreter.get_tensor(output[0]['index'])
  return predictions

In [38]:
(Data, Labels) = load_dataset("../data/uaset_extended_lower_case.csv")

In [39]:
import cv2

# Each image in the A-Z and MNIST digts datasets are 28x28 pixels;
# However, the architecture we're using is designed for 32x32 images,
# So we need to resize them to 32x32

Data = [cv2.resize(image, (32, 32)) for image in Data]
Data = np.array(Data, dtype="float32")

# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]

Data = np.expand_dims(Data, axis=-1)
Data /= 255.0

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

le = LabelBinarizer()
Labels = le.fit_transform(Labels)

counts = Labels.sum(axis=0)

# account for skew in the labeled data
classTotals = Labels.sum(axis=0)
classWeight = {}

# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
  classWeight[i] = classTotals.max() / classTotals[i]
  
(trainX, testX, trainY, testY) = train_test_split(Data,
	Labels, test_size=0.95, stratify=Labels, random_state=42)

In [41]:
# Load the TensorFlow Lite model from file
interpreter = tf.lite.Interpreter(model_path="Ukrainian_OCR.tflite")
interpreter.allocate_tensors()


In [42]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [43]:
BS = 1

In [44]:
from sklearn.metrics import classification_report

# define the list of label names

alphabet = list('абвгґдеєжзиіїйклмнопрстуфхцчшщьюя1234567890№%@,.?:;"!()-\'АБВГҐДЕЄЖЗИІЇЙКЛМНОПРСТУФХЦЧШЩЬЮЯ')
labelNames = [l for l in labelNames]

# Make predictions on test data
predictions = []
for i in range(0, len(testX), BS):
    batch = testX[i:i + BS]
    batch_pre = get_batch_predictions(interpreter, input_details, output_details, batch)
    predictions.append(batch_pre)

predictions = np.vstack(predictions)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))


ValueError: Number of classes, 89, does not match size of target_names, 90. Try specifying the labels parameter

In [ ]:
from imutils import build_montages
from PIL import Image, ImageDraw, ImageFont
import numpy as np

output = ""
images = []
# randomly select a few testing characters

for i in np.random.choice(np.arange(0, len(testY)), size=(66,)):
    
    batch = testX[i][np.newaxis, ...]
    batch_pre = get_batch_predictions(interpreter, input_details, output_details, batch)
    prediction = batch_pre.argmax(axis=1)
    
    label = labelNames[prediction[0]]
    output += label
    image = (testX[i] * 255).astype("uint8")
    color = (0, 255, 0)
    color = (0, 0, 255)
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    img_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype("arial.ttf", size=34)
    draw.text((5, 20), label, font=font, fill=color)
    image = np.array(img_pil)
    images.append(image)
  
montage = build_montages(images, (96, 96), (7, 7))[0]
cv2.imshow('q',montage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
output

'!Фгш-тЄЬопгпТУ.-в!ШПО"ЇчЖй7ТВ(ОПгіі1ьґк8:\'кВЯцт-ЇШМВх№фкьиТіфшю(Ри'